**ДОБАВИЛ ЯЧЕЙКИ ПО ЗАДАНИЮ В ТЕОРЕТИЧЕСКОМ МОДУЛЕ**

**ДОБАВИЛ ЯЧЕЙКИ ПО ЗАДАНИЮ В ТЕОРЕТИЧЕСКОМ МОДУЛЕ**

**ДОБАВИЛ ЯЧЕЙКИ ПО ЗАДАНИЮ В ТЕОРЕТИЧЕСКОМ МОДУЛЕ**

# Проект «Обучение с учителем: качество модели»

**Цель и задачи проекта**

Целью настоящего проекта являлась разработка решения которое позволит персонализировать предложения постоянным клиентам интернет-магазина «В один клик» для увеличения их покупательной активности.
   
   Отчёт магазина за прошлый период показал, что активность покупателей начала снижаться. Привлекать новых клиентов уже не так эффективно: о магазине и так знает большая часть целевой аудитории. Возможный выход — удерживать активность постоянных клиентов. Сделать это можно с помощью персонализированных предложений.

   В начале проекта после подготовки данных к исследованию производилась маркировка уровней финансовой активности постоянных покупателей: «снизилась» и «прежний уровень».

   Собирались данные по клиентам по следующим группам: 
   * признаки, которые описывают коммуникацию сотрудников компании с клиентом;
   * признаки, которые описывают продуктовое поведение покупателя;	
   * признаки, описывающие покупательское поведение клиента;
   * признаки, описывающие поведение покупателя на сайте. 

   После чего проводился корреляционный анализ признаков в количественной шкале в итоговой таблице для моделирования. Был  сделан вывод о наличии мультиколлинеарности.

   Далее применялись модели KNeighborsClassifier, DecisionTreeClassifier, LogisticRegression, SVC для предсказания вероятности снижения покупательской активности клиентов в последующие три месяца. Для каждой модели подбирался как минимум один гиперпараметр. Для установления лучшей модели осуществлялся аргументированный выбор метрики.
   
   Также была оценена важность признаков для лучшей модели и построен график важности с помощью метода SHAP. После чего были сделаны выводы о значимости признаков.
   
   При выполнении сегментации покупателей использовались результаты моделирования и данные о прибыльности покупателей.
   
   В ходе дальнейшей работы над проектом была выбрана произвольная группа покупателей для формирования ей предложений с целью увеличения покупательной способности. Для этого предварительно проводилось графическое и аналитическое исследование групп.
   
   В заключении проекта были сделаны выводы о сегментах:
   * какие сегменты были выбраны для дополнительного исследования;
   * какие предложения были сделаны покупателям и почему.


## Загрузка и изучение данных

### Загрузка библиотек

In [ ]:
# библиотека МО на Python
!pip install --upgrade scikit-learn -q

# библиотека для интерпретации моделей МО
!pip install shap -q

# библиотека для анализа корреляции данных
!pip install phik -q

# библиотека для работы с несбалансированными выборками
!pip install imbalanced-learn

!pip install numpy==1.22.3  # для корректной визуализации по п.7.3.1

!pip install matplotlib==3.5.1 # для корректной визуализации по п.7.3.1

# библиотека визуализации недостающих данных
!pip install missingno

# библиотека с классами и функциями для статистических моделей
!pip install statsmodels

In [ ]:
# ЗАГРУЗКА БИБЛИОТЕК

import pandas as pd                    # анализ и обработка данных
import matplotlib                      # визуализация данных 
import matplotlib.pyplot as plt        # создание простого интерактивного графика 
import numpy as np                     # работа с многомерными массивами
import seaborn as sns                  # визуализаци данных на основе matplotlib
import sklearn                         # машинное обучение
import shap                            # анализ признаков по Шепли
import phik                            # анализ корреляции данных
import missingno as msno               # визуализация пропусков
import warnings                        # игнорирование предупреждений
warnings.filterwarnings('ignore')


# ЗАГРУЗКА МОДЕЛЕЙ
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# ЗАГРУЗКА МЕТРИК
from sklearn.metrics import accuracy_score, precision_score, \
     recall_score, f1_score, roc_auc_score

# ПРЕОБРАЗОВАНИЕ ДАННЫХ ДЛЯ МО
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, \
     OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# ПАЙПЛАЙНЫ, ОПТИМИЗАЦИЯ, ОТБОР ПРИЗНАКОВ
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from phik.report import plot_correlation_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.inspection import permutation_importance

# МОДУЛИ ДЛЯ ПРЕДСТАВЛЕНИЯ ДАННЫХ
from IPython.display import display    #, HTML
pd.set_option('display.float_format', '{:,.2f}'.format)
# display(HTML("<style>.container { width:90% !important; }</style>"))

# УСТАНОВКА КОНСТАНТ
RANDOM_STATE = 42
TEST_SIZE = 0.25

### Загрузка файлов, создание датасетов

#### Загрузка данных о поведении покупателя на сайте магазина, формирование датасета market_file

In [ ]:
# импорт датасета
try:
    #market_file = pd.read_csv('/datasets/market_file.csv', sep=',', decimal='.')
    #market_file.to_csv('market_file.csv', index=False) 
    market_file = pd.read_csv('market_file.csv', sep=',', decimal='.') 
# если не получилось прочитать файл из локальной папки, то загружаем данные из сети
except:
    market_file = pd.read_csv('https://code.s3.yandex.net/datasets/market_file.csv', sep=',', decimal='.')
    # market_file = market_file.to_csv('market_file.csv', index=False)

In [ ]:
market_file.head(3) # первые строки датасета в виде таблицы

In [ ]:
market_file.columns

In [ ]:
# приведение наименований столбцов к единому виду ("змеиная запись" и строчные буквы вместо прописных)
market_file.columns = ['id', 'покупательская_активность', 'тип_сервиса', 'разрешить_сообщать', 
                       'маркет_актив_6_мес', 'маркет_актив_тек_мес', 'длительность', 
                       'акционные_покупки', 'популярная_категория', 'средний_просмотр_категорий_за_визит', 
                       'неоплаченные_продукты_штук_квартал', 'ошибка_сервиса', 'страниц_за_визит']
market_file.head(1) # проверка переименования столбцов

In [ ]:
market_file.info()

#### Загрузка данных с выручкой от покупателей, формирование датасета market_money

In [ ]:
# импорт датасета
try:
    #market_money = pd.read_csv('/datasets/market_money.csv', sep=',', decimal='.')
    #market_money.to_csv('market_money.csv', index=False) 
    market_money = pd.read_csv('market_money.csv', sep=',', decimal='.')
# если не получилось прочитать файл из локальной папки, то загружаем данные из сети
except:
    market_money = pd.read_csv('https://code.s3.yandex.net/datasets/market_money.csv', sep=',', decimal='.')
    # market_money.to_csv('market_money.csv.csv', index=False)

In [ ]:
market_money.head(6) # первые строки датасета в виде таблицы

In [ ]:
market_money.columns

In [ ]:
# приведение наименований столбцов к единому виду ("змеиная запись" и строчные буквы вместо прописных)
market_money.columns = ['id', 'период', 'выручка']
market_money.head(1) # проверка переименования

In [ ]:
market_money.info()

#### Загрузка данных проведенном покупателями времени на сайте, формирование датасета market_time

In [ ]:
# импорт датасета
try:
    #market_time = pd.read_csv('/datasets/market_time.csv', sep=',', decimal='.')
    #market_time.to_csv('market_time.csv', index=False) 
    market_time = pd.read_csv('market_time.csv', sep=',', decimal='.')
# если не получилось прочитать файл из локальной папки, то загружаем данные из сети
except:
    market_time = pd.read_csv('https://code.s3.yandex.net/datasets/market_time.csv', sep=',', decimal='.')
    # market_time.to_csv('market_time.csv.csv', index=False)

In [ ]:
market_time.head(4) # первые строки датасета в виде таблицы

In [ ]:
market_time.columns

In [ ]:
# приведение наименований столбцов к единому виду ("змеиная запись" и строчные буквы вместо прописных)
market_time.columns = ['id', 'период', 'минут']
market_time.head(1) # проверка переименования столбцов

In [ ]:
market_time.info() # вывод общей информации о датасете

#### Загрузка данных по прибыли магазина по каждому покупателю, формирование датасета money

In [ ]:
# импорт датасета
try:
    #money = pd.read_csv('/datasets/money.csv', sep=';', decimal=',')
    #money.to_csv('money.csv', index=False) 
    money = pd.read_csv('money.csv', sep=';', decimal=',')
# если не получилось прочитать файл из локальной папки, то загружаем данные из сети
except:
    money = pd.read_csv('https://code.s3.yandex.net/datasets/money.csv', sep=';', decimal=',')
    # market_time.to_csv('market_time.csv.csv', index=False)

In [ ]:
money.head(3) # первые строки датасета в виде таблицы

In [ ]:
money.columns

In [ ]:
# приведение наименований столбцов к единому виду (без заглавных букв)
money.columns = ['id', 'прибыль']
money.head(1) # проверка переименования столбцов

In [ ]:
money.info() # вывод общей информации о датасете

### Выводы:

**1.** Данные введены корректно, пропуски отсутствуют. Сформированы датасеты **market_file, market_money, market_time, money**.

**2.** Количество строк в датасетах не одинаковое (кроме **market_file** и **money**), но кратно 2 или 3. Обусловлено тем, что в **market_money** и **market_time** на один **id** приходится по 3 или 2 строки с данными, в зависимости от значений периодичности по признаку **'период'**. 

**3.** При первичном ознакомлении с данными сделан вывод, что все признаки в датасетах имеют отношение к содержанию поставленных задач исследования.

**4.** Данные в таблицах соответствуют описанию. Значения признаков всех импортиованных датасетов имеют надлежащие типы данных.

**5.** Названия всех признаков (столбцов) приведены к одному виду ("змеиная запись" и строчные буквы вместо прописных).

## Предобработка данных

### Проверка данных датафреймов на наличие пропусков

In [ ]:
# Функция проверки наличия пропусков в датафреймах
def chec_nan(list_3, list_4):
    for i, k in zip(list_3, list_4):                # цикл с перебором датафреймов при проверке
        print()
        print(f'Проверка датафрейма {k} на наличие пропусков:')
        print(i.isna().sum())                       # проверка очередного датафрейма
    return

In [ ]:
msno.bar(market_file)      #heatmap(market_file)

In [ ]:
# пропусков нет (все белое)
msno.heatmap(market_file)

In [ ]:
list_1 = [market_file, market_money, market_time, money]          # список датафреймов 
list_2 = ["market_file", "market_money", "market_time", "money"]    # список названий датафреймов
chec_nan(list_1, list_2)              # вызов функции поиска пропусков

### Проверка датафреймов на наличие полных дубликатов

In [ ]:
# Функция проверки наличия полных дубликатов в датафреймах
def chec_dupl(list_3, list_4):
    print()
    for i, k in zip(list_3, list_4):                # цикл с перебором датафреймов при проверке
        print(f'Проверка датафрейма {k} на наличие полных дубликатов: {i.duplicated().sum()}')
    return

In [ ]:
list_1 = [market_file, market_money, market_time, money]          # список датафреймов 
list_2 = ["market_file", "market_money", "market_time", "money"]    # список названий датафреймов
chec_dupl(list_1, list_2)             # вызов функции поиска полных дубликатов

### Проверка наличия неявных дубликатов в столбцах датафреймов с категориальными данными

#### Функция поиска неявных дубликатов значений в категориальных столбцах датафреймов

In [ ]:
# Функция поиска неявных дубликатов в столбцах, вывод уникальных значений
def chec_implicit_dupl(df_name):
    # список столбцов датафрейма
    list_1 = df_name.select_dtypes(exclude='number').columns.tolist()
    # цикл с перебором столбцов при проверке
    for i in list_1: 
        print(f'Уникальные значения в столбце "{i}": {df_name[i].unique()}')
    return

#### Проверка датафрейма market_file

In [ ]:
chec_implicit_dupl(market_file)     # вызов функции поиска неявных дубликатов

In [ ]:
# обнаружение неявных дубликатов 'стандартт'
market_file['тип_сервиса'].value_counts()

In [ ]:
# устранение неявных дубликатов в столбце 'тип_сервиса', переименование значений
market_file['тип_сервиса'] = (market_file['тип_сервиса'].replace('стандартт', 'стандарт'))
market_file['тип_сервиса'].unique()     # проверка устранения неявного дубликата

#### Проверка датафрейма market_money

In [ ]:
chec_implicit_dupl(market_money)          # вызов функции поиска неявных дубликатов

#### Проверка датафрейма market_time

In [ ]:
chec_implicit_dupl(market_time)   

In [ ]:
# устранение неявных дубликатов в столбце 'тип_сервиса', переименование значений
market_time['период'] = (market_time['период'].replace('предыдцщий_месяц', 'предыдущий_месяц'))
market_time['период'].unique()     # проверка устранения неявного дубликата

### Выводы:

**1.** Данные в датафреймах полных дубликатов не содержат.

**2.** В датафрейме **market_file** в столбце *'тип_сервиса'* обнаружены неявные дубликаты категориальных значений ('стандартт' - 10 шт.), которые путем приведения к одному значению ('стандарт') устранены.

## Исследовательский анализ данных

Объектом исследования, согласно заданию к проекту, являются данные по клиентам с покупательской активностью не менее трёх месяцев, то есть таких, которые что-либо покупали в этот период. Таким образом, перед проведением исследовательского анализа данных и работой по проекту в целом необходимо "отсечь" данные клиентов, покупательская активность которых не приходится на этот период. Отсев таких клиентов можно осуществить на основании датафрейма **market_money**, который содержит помесячную выручку по покупателям за последние 3 месяца. 

В п.3.1 сначала была осуществлена проверка датафреймов **market_file, market_money, market_time** на наличие дубликатов по столбцу **id** с учетом кратности значений в нем из-за разнесения данных по периодам в столбце **период**. Далее датафрейм **market_money** был преобразован в таблицу **new_market_money** с внесением в созданные столбцы помесячных выручек по покупателям. После чего было проведено присоединение столбцов таблицы **new_market_money** к таблице **market_file** с образованием датафрема **df** (объединение в **df** данных будет использовано в дальнейшем). 

И в завершении объединения к таблице **df** были добавлены данные из датафрейма **market_time**.

### Объединение данных датафреймов market_file,  market_money, market_time в общую таблицу df и отбор данных по покупателям с покупательской активностью не менее 3 меяцев

####  Объединение данных датафреймов market_file, market_money, market_time в общую таблицу df 

In [ ]:
# проверка отсутствия дубликатов 'id' в датасете market_file
market_file['id'].duplicated().sum() 

In [ ]:
# проверка отсутствия дубликатов 'id' в датасете market_money
market_money.value_counts().count()/3 - 1300

In [ ]:
# проверка отсутствия дубликатов 'id' в датасете market_time
market_time.value_counts().count()/2 - 1300  

In [ ]:
# создание на основе датафрейма market_file таблицы df, в которой будут включены данные из других датафреймов
df = market_file

In [ ]:
# назначение индексом признака 'id' в д/ф df
df = df.set_index('id')
df.head(2)

In [ ]:
market_money.head(6) # первые строки датасета market_money в виде таблицы

In [ ]:
# создание из д/ф market_money таблицы со столбцами по выручке за каждый период и проверка
new_market_money = market_money.pivot_table(index='id', columns='период', values='выручка')
new_market_money.head(2)

In [ ]:
# форма таблицы df до объединения
df.shape  

In [ ]:
# форма таблицы new_market_money до объединения
new_market_money.shape

In [ ]:
# присоединение к таблице df таблицы new_market_money с выручкой по 3 последним месяцам
df = df.join(new_market_money)
df.head(2) # контроль результата объединения таблиц

In [ ]:
# форма таблицы после объединения
df.shape  

In [ ]:
# переименование столбцов df для избежания одинаковых наименований при объединении с таблицей market_time
df.columns = ['покупательская_активность', 'тип_сервиса', 'разрешить_сообщать',
              'маркет_актив_6_мес', 'маркет_актив_тек_мес', 'длительность',
              'акционные_покупки', 'популярная_категория', 'сред_просм_кат_за_визит',
              'неоплач_прод_кварт', 'ошибка_сервиса', 'страниц_за_визит', 
              'выручка_пред_месяц', 'выручка_препред_месяц','выручка_тек_месяц']
df.columns # проверка результата переименования

In [ ]:
# первые строки датасета market_time в виде таблицы
market_time.head(4) 

In [ ]:
# создание из д/ф market_time таблицы со столбцами по времени, проведенном на сайте
new_market_time = market_time.pivot_table(index='id', columns='период', values='минут')
new_market_time.head(2)

In [ ]:
# переименование столбцов
new_market_time.columns = ['время_пред_месяц', 'время_тек_месяц']
new_market_time.columns # проверка результата переименования

In [ ]:
# форма new_market_time до присоединения к df
new_market_time.shape

In [ ]:
# присоединение к таблице df таблицы new_market_time
df = df.merge(new_market_time, on='id')
df.head(3)  # проверка результата слияния

In [ ]:
# форма df после присоединения new_market_time
df.shape

In [ ]:
# назначение индексом признака 'id' в д/ф money
money = money.set_index('id')

# присоединение к таблице df таблицы money с прибылью по покупателям по 3 последним месяцам
df = df.join(money)
df.head(2) # контроль результата объединения таблиц

In [ ]:
# форма df после присоединения money
df.shape

#### Отбор данных по покупателям с покупательской активностью не менее 3 меяцев¶

In [ ]:
# удаление в df данных по покупателям с нулевой покупательной активностью 
# хотя бы в одном из 3 месяцев
df = df.loc[(df['выручка_пред_месяц'] > 0) & (df[
    'выручка_препред_месяц'] > 0) & (df['выручка_тек_месяц'] > 0)]

# форма таблицы после удаления данных, не удовлетворяющих условию
df.shape

In [ ]:
# "сброс" индекса
df = df.reset_index(drop =True)

**Результат.** Из таблица **df** удалены три строки, не удовлетворяющие условию.

*Результат.*

**1.** Объединение датасетов произведено с образованием таблицы исходных данных **df**, которая может быть использована для исследовательского анализа данных и дальнейшей работы.

**2.** Из итогового датафрейма удалены три строки по покупателям с покупательской активностью менее 3 мес.

**3.** При ознакомлении с датафреймом **market_file** в признаке **покупательская_активность** было обнаружено последовательное следование сначала строк со значением этого признака 'Снизилась', а затем со значениями 'Прежний уровень'. При разбиении данных на выборки перед началом моделирования необходимо проведение стратификации данных. 

**4.** В завершении данного раздела произведен "сброс" индекса.

### Количественные признаки данных датафрема df

#### Функция графического представления непрерывных количественных признаков

In [ ]:
def cat_numeric(
    cat_column, 
    df_val
    ) -> str:
    """ Функция  графического представления количественных признаков
        (... необходимые детали алгоритма, если нужо ...)
    Args:
        cat_column: столбец датафрейма со значениями по признаку
        df_val: название признака
    Returns:
        plots: Построение гистограммы и боксплота по признаку 
    Rises:
        ValueError: Какие ошибки могут возникнуть во время работы функции 
                    и какой Error или Exception мы можем ожидать
    """   
    
    array_gr = cat_column.to_numpy()
    plt.figure(figsize=(10, 7))
    bins =  plt.hist(array_gr, bins=50)
    plt.vlines(x=array_gr.mean(), colors='red', ymin=bins[0].min(), ymax=bins[0].max(),  label='Среднее значение')
    plt.title(f'Гистограмма распределения признака {df_val}')
    plt.legend()
    plt.ylabel('частота')
    plt.xlabel('значения признака')
    plt.show()
    plt.figure(figsize=(10, 3))
    plt.title(f'Boxplot для признака {df_val}')
    plt.boxplot(array_gr, vert=False)
    plt.xlabel('значения признака')
    plt.show()
    return

In [ ]:
print(cat_numeric.__doc__)

In [ ]:
help(cat_numeric)

#### Функция графического представления дискретных количественных признаков

In [ ]:
def cat_discr_numeric(discr_column):
    plt.figure(figsize=(12, 5))
    sns.countplot(x=discr_column, data=df, hue = 'покупательская_активность')
    plt.title(f'Гистограмма распределения признака "{discr_column}"')
    plt.ylabel('частота')
    plt.xlabel('значения признака')
    plt.show()
    return

#### Ознакомление с общими статистическими характеристиками количественных признаков

In [ ]:
df.head()

In [ ]:
# обзор основных характеристик значений в числовых столбцах
df.describe().T

*Результат:*

**1.** Обнаружено существенное различие значений медианы и средней у признака **'акционные_покупки'**. Возможно присутствуют выбросы. Необходимо обратить внимание в дальнейшем.

**2.** В признаке **'выручка_тек_месяц'** обнаружено аномально высокое максимальное значение равное **106 862 руб.**.

In [ ]:
# количественные признаки
df.select_dtypes(include='number').columns.tolist()

#### Признак "маркет актив 6 мес"

In [ ]:
cat_numeric(df['маркет_актив_6_мес'], '"Кол-во среднемес. маркет. коммуникаций за 6 мес."')

*Результат.* Отмечаются выбросы слева от медианы.

#### Признак "маркет_актив_тек_мес"

In [ ]:
cat_discr_numeric('маркет_актив_тек_мес')

#### Признак "длительность"

In [ ]:
cat_numeric(df['длительность'], '"Кол-во дней после регистрации покупателя"')

#### Признак "акционные_покупки"	

In [ ]:
cat_numeric(df['акционные_покупки'], '"Среднемес. доля покупок по акции за 6 мес."')

In [ ]:
sns.histplot(data = df, x = 'акционные_покупки', hue='покупательская_активность', bins=50, multiple='dodge');

*Результат.* Выявлена кластеризация данных признака по двум категориям (правый кластер не  выброс). В связи с чем, на основе этого признака был создан дополнительный бинарный признака признак **'акц_пок_бин'**, **0** признаку присваивался при значениях признака **'акционные_покупки'** ниже **0.6** и **1** - при превышении этого предела. Однако, при построении матрицы корреляции в п.4.2 эти два признака оказались **мультиколлениарными**, причем связь бинарного признака **'акц_пок_бин'** с целевым (0,49) оказалась слабее связи признака **'акционные_покупки'** (0,51). В связи с этим, признак **'акц_пок_бин'** был исключен из датафрейма **df**.

#### Признак "сред_просм_кат_за_визит"	

In [ ]:
cat_discr_numeric('сред_просм_кат_за_визит')

#### Признак "неоплач_прод_кварт"	

In [ ]:
cat_discr_numeric('неоплач_прод_кварт')

#### Признак "ошибка_сервиса"	

In [ ]:
cat_discr_numeric('ошибка_сервиса')

#### Признак "страниц_за_визит"	

In [ ]:
cat_discr_numeric('страниц_за_визит')

#### Признак "выручка_пред_месяц"	

In [ ]:
cat_numeric(df['выручка_пред_месяц'], '"Выручка по клиентам в предыдущий месяц"')

#### Признак "выручка_препред_месяц"

In [ ]:
cat_numeric(df['выручка_препред_месяц'], '"Выручка по клиентам в препредыдущий месяц"')

#### Признак "выручка_тек_месяц"	

In [ ]:
cat_numeric(df['выручка_тек_месяц'], '"Выручка по клиентам в текущем месяце"')

**Результат.** Обнаружено аномально высокое значение признака "выручка_тек_месяц"

In [ ]:
# определение значения выброса, превышающего 100000 руб.
df['выручка_тек_месяц'].sort_values(ascending=False).head(2)

In [ ]:
# удаление аномального значения заменой его на медианное
df.loc[df['выручка_тек_месяц'] >= 20000, 'выручка_тек_месяц'] = df.loc[df['выручка_тек_месяц'] < 20000, 'выручка_тек_месяц'].median()
df.loc[29, 'выручка_тек_месяц']  # проверка замены

In [ ]:
# после удаления аномалии
cat_numeric(df['выручка_тек_месяц'], '"Выручка по клиентам в текущем месяце"')

#### Признак "время_пред_месяц"

In [ ]:
cat_discr_numeric('время_пред_месяц')

#### Признак "время_тек_месяц"	

In [ ]:
cat_discr_numeric('время_тек_месяц')

#### Признак "прибыль"	

In [ ]:
cat_numeric(df['прибыль'], '"Прибыль"')

#### Построение гистограмм и диаграммы размаха для количественных признаков датафрейма df

**Функция для построения обзорных гистограмм и диаграмм размаха для количественных признаков датафрейма df**

In [ ]:
def hist_all_num(list_num):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(13,5))
    df[list_num].plot(kind='hist', bins=30, alpha=0.5, ax=axes[0])
    axes[0].set_title('Гистограмма распределения')
    df[list_num].plot(kind='box', ax=axes[1], rot=60)
    axes[1].set_title('Разброс значений признаков')
    plt.ylabel('значения признаков')
    plt.show()
    return

**Обзорная гистограмма и диаграмма размаха для количественных признаков датафрейма df, за исключением признаков по выручке и длительности**

In [ ]:
# признаки для гистограммы и диаграммы размаха
num_col_names = df.select_dtypes(include='number').columns.tolist()
num_col_names = ['маркет_актив_6_мес', 'маркет_актив_тек_мес', 'акционные_покупки', 'сред_просм_кат_за_визит', 
                 'неоплач_прод_кварт', 'ошибка_сервиса', 'страниц_за_визит', 'время_пред_месяц', 
                 'время_тек_месяц', 'прибыль']

# вызов функции
hist_all_num(num_col_names)

**Обзорная гистограмма и диаграмма размаха для признаков по длительности и выручке датафрейма df**

In [ ]:
# признаки для гистограммы и диаграммы размаха
num_col_names = ['длительность', 'выручка_пред_месяц', 'выручка_препред_месяц', 'выручка_тек_месяц']

# вызов функции
hist_all_num(num_col_names)

*Результат.* Необходимость проведения масштабирования данных.

### Категориальные признаки

#### Функция графического представления категориальных признаков

In [ ]:
# функция  графического представления категориальных признаков

def cat_category(cat_column):
    if cat_column == 'популярная_категория': 
        rot_x = 45 
    else: 
        rot_x = 0
    plt.figure(figsize=(6, 2))
    df[cat_column].value_counts(normalize=True).plot(kind='bar', rot=rot_x)
    plt.title(f'Гистограмма распределения признака "{cat_column}"')
    plt.ylabel('доля категорий')
    plt.xlabel('значения признака')
    plt.ylabel('частота')
    plt.show()
    return

In [ ]:
# категориальные столбцы
df.select_dtypes(exclude='number').columns.tolist()

#### Целевой признак "покупательская_активность"

In [ ]:
cat_category('покупательская_активность')

In [ ]:
# замена значений категорильного столбца:
# 1 - при снижении покупательской активности
# 0 - при сохранении покупательской активности
# df['покупательская_активность'] = df['покупательская_активность'].apply(lambda x: 1 if x=='Снизилась' else 0)

#### Признак "тип_сервиса"

In [ ]:
cat_category('тип_сервиса')

#### Признак "разрешить_сообщать"

In [ ]:
cat_category('разрешить_сообщать')

#### Признак "популярная_категория"

In [ ]:
# визуализация частот признака "популярная_категория"
(
    df['популярная_категория']
    .value_counts(normalize=True)
    .sort_values(ascending=True)
    .to_frame()
    .plot(kind = 'barh', grid=True, fontsize = 12, figsize = (12,4))
).set_title('Гистограмма распределения признака "популярная_категория"',  fontsize = 16, color = 'Black')
plt.xlabel('доля категории', fontsize=15)
plt.ylabel('названия категорий', fontsize=15)
plt.show()

### Выводы:

**1.** Перед исследовательским анализом данных было произведено их объединение в один датафрейм **df**.  При этом, в ходе проверки клиентов с покупательской активностью не менее трёх месяцев, то есть таких, которые что-либо покупали в этот период, вывлено не было.

**2.** При ознакомлении с датафреймом **market_file** в признаке **покупательская_активность** было обнаружено последовательное следование сначала строк со значением этого признака 'Снизилась', а затем со значениями 'Прежний уровень'. Причем отмечается дисбаланс классов. При разбиении данных на выборки перед началом моделирования необходимо проведение стратификации данных.

**3.** При ознакомлении с данными признака **"маркет актив 6 мес"** (п.3.2.3), **"неоплач_прод_кварт"** (п.3.2.8) обнаружены данные, выходящие за пределы боксплота, но не являющиеся аномалиями.

**4.** Произведено преобразование дискретного количественного признака **маркет_актив_тек_мес** в порядковый категориальный.

**5.** В данных по признаку **"выручка_тек_месяц"** (п.3.2.13) обнаружен аномальный выброс со значением **106,862.20 руб.**, который был удален путем замены на медианное значение по выборке, а именно на **5179.95 руб.**

**6.** Построенные гистограмма и диаграмма размаха для количественных признаков датафрейма **df** позволяют сделать вывод о необходимости проведения масштабирования данных по признакам перед проведением моделирования.

**7.** Гистограммы распределения категориальных признаков выявляют дисбаланс их классов.

**8.** По целевому признаку **'покупательская_активность'** необходимо проведение стратификации данных ввиду существенного количественного отличия классов **"снизилась"** и **"прежний уровень"** (соотношение 2 к 3). 

In [ ]:
data_df = df[['маркет_актив_6_мес', 'акционные_покупки', 'тип_сервиса', 'длительность', 'прибыль', 'покупательская_активность']]
sns.pairplot(data_df, hue = 'покупательская_активность');

In [ ]:
g = sns.PairGrid(data_df, hue='покупательская_активность')
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend();

In [ ]:
sns.displot(data = df, x = 'длительность', hue='покупательская_активность', bins=20, multiple='dodge');  
# histplot, discrete=True, multiple='dodge'stat='density'

In [ ]:
g = sns.FacetGrid(data_df, col='тип_сервиса', hue='покупательская_активность')
g.map(sns.scatterplot, 'акционные_покупки', 'прибыль', alpha=.3)
g.add_legend();

## Корреляцилнный анализ признаков

### Отбор количественных непрерывных признаков для корреляционной матрицы

In [ ]:
# dывод наименований всех признаков
df.columns

In [ ]:
# количественные непрерывные признаки
interval_cols=['маркет_актив_6_мес', 'длительность', 'акционные_покупки',
               'выручка_препред_месяц', 'выручка_пред_месяц', 'выручка_тек_месяц']

### Матрица коэффициентов корреляции для целевого и входных признаков

In [ ]:
# расчет матрицы корреляции
phik_overview = df.phik_matrix(interval_cols=interval_cols)

# вывод матрицы корреляции
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title='Коэффициенты корреляции между признаками',
    fontsize_factor=1.5,
    figsize=(18, 15)
) 

### Выводы:

**1.** Выявлена умеренная корреляция между целевым признаком **'покупательская_активность'** и признаками **'неоплач_прод_кварт'** (0,40), **'популярная_категория'** (0,30).

**2.** Заметная корреляция установлена между целевым признаком и признаками **'время_тек_месяц'** (0,57) и **'время_пред_месяц'** (0,60), **'выручка_препред_месяц'** (0.50), **'сред_просм_кат_за_визит'** (0.54), **'акционные_покупки_бин'** (0.49), **'маркет актив 6 мес'** (0.55).

**3.** Высокая корреляция отмечена между целевым признаком и признаком **'страниц_за_визит'** (0.75).

**4.** **Мультиколлениарность** признаков не выявлена.

**СОВЕТ 1. ДОПОЛНИТЕЛЬНЫЕ МАТРИЦЫ КОРРЕЛЯЦИИ**

In [ ]:
# расчет матрицы корреляции для 'покупательская_активность' == "Прежний уровень"
phik_overview = df.query('покупательская_активность == "Прежний уровень"').phik_matrix(interval_cols=interval_cols)

# вывод матрицы корреляции
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title='Коэффициенты корреляции между признаками',
    fontsize_factor=1.5,
    figsize=(18, 15)
) 

**СОВЕТ 2. ПРОВЕРКА НА МУЛЬТИКОЛЛИНЕАРНОСТЬ С ПОМОЩЬЮ VIF**

In [ ]:
X_df = df.copy()

In [ ]:
# категориальные признаки
ohe_columns = [
    'популярная_категория'   
]

# упорядоченные признаки
ord_columns = [
    'тип_сервиса',
    'разрешить_сообщать',
    'покупательская_активность'
]

# количественные признаки
num_columns = [
    'маркет_актив_6_мес',
    'маркет_актив_тек_мес',
    'длительность',
    'акционные_покупки',
    'сред_просм_кат_за_визит',
    'неоплач_прод_кварт',
    'ошибка_сервиса',
    'страниц_за_визит',
    'выручка_препред_месяц',
    'выручка_пред_месяц',
    'выручка_тек_месяц',
    'время_пред_месяц',
    'время_тек_месяц',
    'прибыль'
]

In [ ]:
# SimpleImputer + OHE
ohe_pipe_1 = Pipeline(
    [
        (
            'simpleImputer_ohe', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ohe', 
            OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
        )
    ]
) 

In [ ]:
# SimpleImputer + ORD + SimpleImputer
ord_pipe_1 = Pipeline(
    [
        (
            'simpleImputer_before_ord', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',  
            OrdinalEncoder(
                categories=[
                    ['премиум', 'стандарт' ],
                    ['да', 'нет'],
                    ['Прежний уровень', 'Снизилась']
                ], 
                handle_unknown='use_encoded_value', unknown_value=np.nan
            )
        ),
        (
            'simpleImputer_after_ord', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
) 

In [ ]:
data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe_1, ohe_columns),
        ('ord', ord_pipe_1, ord_columns),
        ('num', MinMaxScaler(), num_columns)
    ], 
    remainder='passthrough'
)

In [ ]:
X_df = pd.DataFrame(data_preprocessor.fit_transform(df), columns=data_preprocessor.get_feature_names_out())

In [ ]:
X_df.columns

In [ ]:
# "укорачивание" названий признаков в преобразованных тренировочной и тестовой выборках
new_names = ['косметика_и_аксесуары', 'кухонная_посуда', 'бытовая_техника_и_эл_ка', 
             'тех_красоты_и_здоровья', 'товары_для_детей', 'тип_сервиса_стандарт', 
             'разрешить_сообщать', 'покупательская_активность', 
             'маркет_актив_6_мес', 'маркет_актив_тек_мес', 'длительность', 
             'акционные_покупки', 'сред_просм_за_визит', 'отказов_за_квартал', 
             'ошибка_сервиса', 'страниц_за_визит', 'выручка_препред_месяц', 
             'выручка_пред_месяц', 'выручка_тек_месяц', 'время_пред_месяц', 
             'время_тек_месяц', 'прибыль']

X_df.columns = new_names

In [ ]:
X_df.head(2)

In [ ]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data['ПРИЗНАКИ     '] = X_df.columns

# calculating VIF for each feature
vif_data['VIF'] = [variance_inflation_factor(X_df.values, i) for i in range(len(X_df.columns))]

print(vif_data)

**СОВЕТ 1. Cвязь между выручка текущая - выручка за прошлый с помощью scatter_plot.**

In [ ]:
data_df = df[['выручка_тек_месяц', 'выручка_пред_месяц', 'выручка_препред_месяц', 'покупательская_активность']]
g = sns.PairGrid(data_df, hue='покупательская_активность')
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend();

In [ ]:
data_cut_1 = X_df[['тип_сервиса_стандарт', 'разрешить_сообщать', 'покупательская_активность']]
data_cut_1['выручка_препред_пред'] = X_df['выручка_препред_месяц'] + X_df['выручка_пред_месяц']
g = sns.PairGrid(data_cut_1, hue='покупательская_активность')
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend();

In [ ]:
data_cut_2 = X_df[['бытовая_техника_и_эл_ка', 'тех_красоты_и_здоровья', 'покупательская_активность']]
data_cut_2['выручка_препред_пред'] = X_df['выручка_препред_месяц'] + X_df['выручка_пред_месяц']
g = sns.PairGrid(data_cut_2, hue='покупательская_активность')
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend();

In [ ]:
data_df_1 = df[['покупательская_активность']]
data_df_1['выручка_пред_препред'] =  X_df['выручка_пред_месяц'] - X_df['выручка_препред_месяц']
data_df_1['выручка_тек_пред'] =  X_df['выручка_тек_месяц'] - X_df['выручка_пред_месяц']
g = sns.PairGrid(data_df_1, hue='покупательская_активность')
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend();

## Создание пайплайна, определение лучшей модели

### Подготовка данных

При проведении разбиения данных на выборки была проведена стратификация, о необходимости которой указано в **выводе 2 п.3.4**. Для логистической модели изначально была предусмотрена **L1-регуляризация**.

#### Разбиение исходных данных (датафрейм df) на выборки

In [ ]:
# обзор исходного датафрейма
df.head(2)

In [ ]:
X = df.drop(columns='покупательская_активность') # выборка исходных признаков
y = df['покупательская_активность']              # выборка целевого признака

# деление данных на тренировочные и тестовые, стратификация данных
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y)

In [ ]:
X_test[['прибыль']].head()

#### Кодирование целевого признака с помощью LabelEncoder

In [ ]:
# объявление трансформера категорильного признака (y_train)
label_encoder = LabelEncoder()

# преобразование значений категориального признака в 0 и 1
y_train = label_encoder.fit_transform(y_train.values)
y_test = label_encoder.transform(y_test.values)

# проверка кодирования
y_test

In [ ]:
# проверка, что 0 - 'Прежний уровень', 1 - 'Снизилась'
pred_classes = [0, 1, 0, 1, 1]
print(label_encoder.inverse_transform(pred_classes)) 

In [ ]:
# сохранение в отдельных датафреймах значений признака 'прибыль' для использования в дальнейшем
money_train = X_train[['прибыль']].reset_index(drop=True)
money_test = X_test[['прибыль']].reset_index(drop=True)

# удаление признака 'прибыль' перед моделированием
X_train = X_train.drop(columns='прибыль')
X_test = X_test.drop(columns='прибыль')

money_test.head(3) # контроль преобразования

####  Группировка по типам исходных данных

In [ ]:
# категориальные признаки
ohe_columns = [
    'популярная_категория'   
]

# упорядоченные признаки
ord_columns = [
    'тип_сервиса',
    'разрешить_сообщать'
]

# количественные признаки
num_columns = [
    'маркет_актив_6_мес',
    'маркет_актив_тек_мес',
    'длительность',
    'акционные_покупки',
    'сред_просм_кат_за_визит',
    'неоплач_прод_кварт',
    'ошибка_сервиса',
    'страниц_за_визит',
    'выручка_препред_месяц',
    'выручка_пред_месяц',
    'выручка_тек_месяц',
    'время_пред_месяц',
    'время_тек_месяц'
]

### Построение и применение пайплайна

#### OHE-кодирование

In [ ]:
# SimpleImputer + OHE
ohe_pipe = Pipeline(
    [
        (
            'simpleImputer_ohe', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ohe', 
            OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
        )
    ]
) 

#### ORD-кодирование

In [ ]:
# SimpleImputer + ORD + SimpleImputer
ord_pipe = Pipeline(
    [
        (
            'simpleImputer_before_ord', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',  
            OrdinalEncoder(
                categories=[
                    ['премиум', 'стандарт' ],
                    ['да', 'нет']
                ], 
                handle_unknown='use_encoded_value', unknown_value=np.nan
            )
        ),
        (
            'simpleImputer_after_ord', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
) 

#### Масштабирование количественных признаков и объединение подготовительных процедур

In [ ]:
data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('ord', ord_pipe, ord_columns),
        ('num', StandardScaler(), num_columns)
    ], 
    remainder='passthrough'
)

#### Итоговый пайплайн для обучения модели

In [ ]:
# preprocessor__num = preprocessor_ + _num:
# preprocessor__num — пайплайн, в котором находится шаг масштабирования
# num — шаг масштабирования с методом и списком столбцов
#'passthrough' — вариант без масштабирования

# итоговый пайплайн: подготовка данных и модель
pipe_final= Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_distributions = [
    # словарь для модели KNeighborsClassifier() 
#    {
        # название модели
#        'models': [KNeighborsClassifier()],
        # указываем гиперпараметр модели n_neighbors
#        'models__n_neighbors': range(10, 20),
        # указываем список методов масштабирования
#        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
#    },
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 9),
        'models__max_features': range(2, 4),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    # словарь для модели SVC()
    {
        'models': [SVC(kernel='linear', random_state=RANDOM_STATE, class_weight='balanced', probability=True)],
        'models__C': [0.1, 0.2, 0.3],
        'models__gamma': [0.3, 0.4, 0.5],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  

    },
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE,
            class_weight='balanced',
            solver='saga',                   
            penalty= 'l1'
        )],
        'models__C': range(1,6), # [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6] range(1,10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    }
]

In [ ]:
# cловарь с двумя метриками:
# - roc-auc с именем roc_auc_score;
# - recall с именем recall_score;
# - f1 с именем f1__score.
scoring = {
    'roc_auc_score': 'roc_auc',
    'recall_score': 'recall',
    'f1__score': 'f1'
} 

# перебор вариантов масштабирования и гиперпараметров для моделей из pipe_final
randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_distributions=param_distributions,
    cv = 5, 
    scoring = scoring,
    refit ='roc_auc_score',
    random_state = RANDOM_STATE,
    n_jobs = -1
)

#### Применение пайплайна к тренировочным данным

In [ ]:
# обучение модели на тренировочной выборке
randomized_search.fit(X_train, y_train) 

#### Наилучшая модель и ее метрика по итогам использования пайплайна

In [ ]:
print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_) 

In [ ]:
print(f'Параметры лучшей модели: {randomized_search.best_params_}')

In [ ]:
print(f'Метрика ROC-AUC лучшей модели: {randomized_search.best_score_:.3f}')

In [ ]:
# cтрока в таблице для лучшей модели
print('Номер строки с параметрами лучшей модели: ', randomized_search.best_index_)

In [ ]:
# транспонированная (для удобства) таблица с результатами работы пайплайна
print(pd.DataFrame(randomized_search.cv_results_).T)

#### Извлечение преобразованных пайплайном выборок

In [ ]:
# обучение и преобразование препроцессором данных X_train
data_preprocessor.fit_transform(X_train)

In [ ]:
# получение преобразованной пайплайном тренировочной выборки
train_new = pd.DataFrame(randomized_search.best_estimator_['preprocessor'].fit_transform(X_train), columns=data_preprocessor.get_feature_names_out())
train_new.head(2)

In [ ]:
# получение преобразованной пайплайном тестовой выборки
test_new = pd.DataFrame(randomized_search.best_estimator_['preprocessor'].fit_transform(X_test), columns=data_preprocessor.get_feature_names_out())
test_new.head(2)

In [ ]:
# "укорачивание" названий признаков в преобразованных тренировочной и тестовой выборках
new_names = ['косметика_и_аксесуары', 'кухонная_посуда', 'бытовая_техника_и_эл_ка', 
             'тех_красоты_и_здоровья', 'товары_для_детей', 'тип_сервиса_стандарт', 
             'разрешить_сообщать', 'маркет_актив_6_мес', 'маркет_актив_тек_мес', 
             'длительность', 'акционные_покупки', 'сред_просм_за_визит', 
             'отказов_за_квартал', 'ошибка_сервиса', 'страниц_за_визит', 
             'выручка_препред_месяц', 'выручка_пред_месяц', 'выручка_тек_месяц', 
             'время_пред_месяц', 'время_тек_месяц']

train_new.columns = new_names
test_new.columns =  new_names

In [ ]:
train_new.columns # проверка переименований

#### Расчет метрик качества лучшей модели на тестовых данных

In [ ]:
# применение обученной модели на тестовой выборке
y_test_pred_new = randomized_search.best_estimator_['models'].predict(test_new)
y_test_proba_new = randomized_search.best_estimator_['models'].predict_proba(test_new)[:,1]
print('МЕТРИКИ ДЛЯ ТЕСТОВОЙ ВЫБОРКИ:')
print(f'Метрика ROC-AUC: {roc_auc_score(y_test, y_test_proba_new):.3f}')
print(f'Метрика accuracy:  {accuracy_score(y_test, y_test_pred_new):.3f}')
print(f'Метрика precision:  {precision_score(y_test, y_test_pred_new):.3f}')
print(f'Метрика recall:  {recall_score(y_test, y_test_pred_new):.3f}')
print(f'Метрика f1:  {f1_score(y_test, y_test_pred_new):.3f}')

### Выводы:

**1.** Производилось обучение четырех моделей: **DecisionTreeClassifier(), LogisticRegression() и  SVC()**. Для каждой из них подбирался как минимум один гиперпараметр. Было произведено большое количество вариаций гиперпараметров, в результате чего была подобрана модель на основе логистической регресии, обладающая наилучшими показателями метрик качества. 

**2.** Лучшая модель на тестовых выборках обладает лучшей метрикой качества по сравнению с полученными на основе тренировочных выборок с валидацией.

**3.** Для автоматизированного поиска оптимальных гиперпараметров использовался модуль **RandomizedSearchCV**, так как позволяет обеспечить оперативный поиск решений при большом количестве вариаций значений гиперпараметров.

**4.** При подборе гиперпараметров использовалась метрика **ROC-AUC**, как характеризующая качество модели в целом, которая показывает зависимость TPR и FPR. Оценивает работу модели вне зависимости от выбранного порога вероятности, поэтому она всегда даёт адекватную оценку качества модели. Работает и при несбалансированных классах.

**5.** Параметры лучшей модели: **{'preprocessor__num': MinMaxScaler(), 'models__C': 2, 'models': LogisticRegression(class_weight='balanced', penalty='l1', random_state=42, solver='saga')}**.

**6.** Выбор пайплайном в качестве лучшей логистической модели представляется вполне объяснимой ввиду структуры исходных данных для анализа. Большинство признаков количественные. Между целевым признаком и шестью входящими признаками имеется умеренная линейная связь, а с еще одним признаком заметная связь.

**7.** Метрики качества итоговой модели на тестовой выборке:

* ROC-AUC: 0.922
* accuracy:  0.855
* precision:  0.794
* recall:  0.839
* f1:  0.816

## Анализ важности признаков

### Анализ важности признаков по весовым коэффициентам логистической модели

#### Визуализация коэффициентов логистической модели

In [ ]:
# коэффициенты логистической регрессии
coefficients = randomized_search.best_estimator_['models'].coef_[0]

# формирование таблицы важности признаков
feature_importance = pd.DataFrame(
    {
        'Feature': train_new.columns, 
        'Importance': np.abs(coefficients)
    })

# визуализация весов коэффициентов весов оптимизированной логистической модели
feature_importance = feature_importance.sort_values('Importance', ascending=True)
feature_importance.plot(x='Feature', y='Importance', kind='barh', grid=True, figsize=(8, 10))
plt.title(f'Коэффициенты оптимизированной логистической модели')
plt.xlabel('весовые значения'); 

#### Выделение важных признаков путем максимизации значений метрики ROC-AUC при валидации тренировочной выборки

In [ ]:
# вывод метрики при кросс-валидации
print()
print('Расчет метрики ROC-AUC при разных значениях feature_importance["Importance"]:')
print()

# цикл по вычислению метрики ROC-AUC при разных порогах feature_importance["Importance"]
# вывод результатов расчета метрик для разных пороговых значений
for i in range(0,12):
    # формирование списка признаков более i
    features_to_drop = feature_importance[
        feature_importance['Importance'] <= 0.2*i
    ]['Feature'].to_list()

    # удаление признаков из датасета
    X_train_less_features = train_new.drop(columns=features_to_drop)

    # подсчет метрики при кросс-валидации
    score = cross_val_score(
        randomized_search.best_estimator_['models'],                     
        X_train_less_features,
        y_train,
        scoring='roc_auc',
        n_jobs=-1
    ).mean()

    # вывод метрики при заданной i-ой важности признаков
    print(f'порог {(i*0.2):.1f}: {score:.3f}')

In [ ]:
# отбор признаков по оптимальному значению порога важности признака 
# и его весовому коэффициенту регрессии
print('РЕЗУЛЬТАТ ОТБОРА ПРИЗНАКОВ:')
print(feature_importance.query('Importance >= 0.2').sort_values('Importance', ascending=False))

*Результат.*

Фильтрация признаков позволила позволила отобрать 14 признаков при оптимальном пороговом значении важности признака **0.4**: **'страниц_за_визит', 'отказов_за_квартал', 'сред_просм_за_визит', 'время_пред_месяц', 'маркет актив 6 мес', 'акционные_покупки', 'время_тек_месяц', 'выручка_препред_месяц', 'бытовая техника и эл-ка', 'выручка_пред_месяц', 'тех красоты и здоровья', 'выручка_тек_месяц', 'длительность', 'ошибка_сервиса'.**

### Отбор признаков на основе статистического теста SelectKBest

#### Отбор на основе оценки взаимной информации между признаками

In [ ]:
print('ОТОБРАННЫЕ ПРИЗНАКИ:')
#  тест на основе линейной зависимости между входным и целевым признаком
selector = SelectKBest(mutual_info_classif, k=14) 
selector.fit(train_new, y_train)
features_names = train_new.columns[selector.get_support(indices=True)]
print('Тест на основе линейной зависимости:', list(features_names))

*Результат.*

Совпадение по 13 признакам по сравнению с п.6.1.2.

#### Отбор признаков по выручке от продаж покупателям в текущем месяце

Здесь проводился эксперимент с использованием следующего критерия.

При отборе признаков с помощью лучшей модели целесообразной представляется минимизация ошибки второго рода (**FN**), то есть случаев ошибочных предсказаний стабильной активности покупателей, но при эжтом, на самом деле, снижения активности покупателей вопреки прогнозу.

Количество ошибочных предсказаний равно **FN**. Этому показателю соответсвует выручка за текущий месяц **fn** по данной группе покупателей.

Ниже произведен отбор признаков (формирование их состава) на основе минимизации сумм потенциальных потерь, соответствующих ошибке **FN**.

In [ ]:
# списки для итогового вывода результатов
roc_auc = []
money = []
y_train = pd.DataFrame(data=y_train, columns=['покупательская_активность'])

# параметры для вывода результатов
min_select = 100000000000
features_names_select = []
i_select=0

# заголовки столбцов
print('количество   roc-auc     доход за тек. мес.')
print('признаков                    усл.ед.')
print()

# датафреймы для анализа из п.5.2.7
X_train_select =  train_new.copy()

# лучшая модель
model = randomized_search.best_estimator_['models']

# циклический отбор признаков по максимальной выручке за 3 мес.
for i in range(len(X_train_select.columns)):
    # объявление селектора 
    # для подбора разного количества признаков за счёт цикла
    selector = SelectKBest(f_classif, k=i+1)
    selector.fit(X_train_select, y_train)
    
    # сохранение названия признаков в переменную features_names
    features_names = selector.get_feature_names_out()

    # формирование датафреймов с новым количеством признаков
    X_train_new = X_train_select[list(features_names)]
    
    # инициализация и обучение модели
    model.fit(X_train_new, y_train)
    
    # подсчет метрики при кросс-валидации
    score = cross_val_score(
        model,                     
        X_train_new,
        y_train,
        scoring='roc_auc',
        n_jobs=-1
    ).mean()
    
    # расчет метрики roc-auc для i-ой совокупности признаков
    roc_auc_new = score

    # создание таблицы предсказаний для подсчёта прогнозов и суммарной выручки
    predicts = X_train_select[['выручка_тек_месяц']].copy() 
    predicts['progn'] = cross_val_predict(model, X_train_new, y_train, n_jobs=-1)
    predicts['y_train'] = y_train['покупательская_активность']            
    
    # подсчет выручки по клиентам, активность по которым снизилась при предсказанной стабильной активности
    fn = predicts[(predicts['progn']==0)&(predicts['y_train']==1)]['выручка_тек_месяц'].sum()
    
    # выбор оптимального количества признаков и их перечня
    if  fn < min_select:
        min_select = fn
        features_names_select = features_names
        i_select = i + 1
    
    # заполнение списков для вывода результатов
    roc_auc.append(roc_auc_new)
    money.append(fn)
    
    # вывод результаатов
    if i+1 < 10:
        print(f'    {i+1}         {roc_auc_new:.3f}            {fn:.0f}')
    else:
        print(f'    {i+1}        {roc_auc_new:.3f}            {fn:.0f}')
        
print()
print(f'Отобрано {i_select} признаков:')
print()
print(features_names_select)

*Результат.*

Отобраны 9 признаков, также входящие в состав признаов, отобранных в п.6.1.2 и п.6.2.1. Данному набору признаков при минимальной выручке (**34**) соответствует максимальный показатель метрики ROC-AUC (**0,902**).

### Анализ значимости признаков по методу SHAP

####  Диаграмма общей значимости признаков

In [ ]:
model = randomized_search.best_estimator_['models']
explainer = shap.LinearExplainer(model, test_new)
shap_values = explainer(test_new)

plt.title('Значимость признаков по методу SHAP', fontsize=20)
plt.ylabel('признаки', fontsize=16)
shap.plots.bar(shap_values, max_display=20) 
plt.show()

#### Анализ значимости признаков с их расширеной визуализацией

In [ ]:
plt.title('Анализ значимости признаков с их расширеной визуализацией', fontsize=20)
plt.ylabel('признаки', fontsize=16)
shap.plots.beeswarm(shap_values, max_display=20)
plt.show()

#### Общие заключения по графику значимости признаков SHAP

**1.** Существенное время, проведимое покупателями на сайте интернет-магазина ('сред_просм_за_визит', 'страниц_за_визит', 'время_пред_месяц', 'время_тек_месяц'), говорит о привлекательности предложений и способствует сохранению активности покупателей.

**2.** Увеличение количества отказов от покупок обуславливает снижение интереса клиентов к интернет-ресурсу.

**3.** Выделяется группа покупателей с повышенным участием в акциях компании, но, в то же время, сокращающие свои покупки. Поведение данной категории клиентов целесообразно проанализировать.

**4.** Активная маркетинговая политика компании однозначно положительно сказывается на активности пользователей сайта. Снижение маркетинговых коммуникаций вызывает спад заинтересованности.

**5.** Отмечаются незначительная по составу группа клиентов, заинтересованных в регулярном (сохраняется интерес) приобретении бытовой техники и электронии, и группа, покупающая технику для красоты и здоровья.

### Выводы:

**1.** При анализе весовых коэффициентов логистической модели (п.6.1) важными классифицированы следующие 14 признаков:
* **'страниц_за_визит'** - среднее количество страниц, которые просмотрел покупатель за один визит на сайт за 3 месяца;
* **'отказов_за_квартал'** - общее число неоплаченных товаров в корзине за последние 3 месяца;
* **'сред_просм_за_визит'** -  сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца;
* **'время_пред_месяц'** - время на сайте в минутах, зафиксированное за предыдущий месяц;
* **'маркет актив 6 мес'** -  среднемес. значение маркетинг. коммуникаций, которое приходилось на покупателя за 6 месяцев;
* **'акционные_покупки'** - среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев;
* **'время_тек_месяц'** - время на сайте в минутах, зафиксированное за предыдущий месяц;
* **'выручка_препред_месяц'** - сумма выручки за препредыдущий месяц;
* **'бытовая техника и эл-ка'** - популярная категория "бытовая техника и электроника";
* **'выручка_пред_месяц'** - сумма выручки за предыдущий месяц;
* **'тех красоты и здоровья'** - 'популярная категория "техника для красоты и здоровья";
* **'выручка_тек_месяц'** - сумма выручки за текущий месяц;
* **'длительность'** - значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте;
* **'ошибка_сервиса'** - число сбоев, которые коснулись покупателя во время посещения сайта.

Остальные признаки при добавлении с исследуемую выборку ухудшают качество модели (метрика **'ROC-AUC'**) и могут быть признаны малозначимыми, а именно: 

**'тип_сервиса_стандарт', 'товары_для_детей', 'косметика и аксесуары', 'маркет_актив_тек_мес', 'разрешить_сообщать_нет', 'кухонная_посуда'**.

**2.** Схожий по большинству признаков результат отбора наиболее важных признаков дает метод **SelectKBest** (совпадение по 13 признакам с п.1 выводов, п.6.4).

**3.** В ходе работы над проетом в состав количественных признаков были включены дополнительные два признака, представляющие собой разность выручек по текущему и предыдущему, предыдущему и препредыдущему месяцам. Но, по итогам моделирования, добавление этих признаков ухудшило качество модели. При этом пайплайн отобрал ту же логистическую модель с теми же параметрами. Данные признаки были исключены из анализа. 

## Сегментация покупателей

При работе над проектом произведен анализ следующих сегментов покупателей:

* с максимальной долей покупок по акции и высокой вероятностью снижения покупательской активности;
*  покупающих только технику (товары с длинным жизненным циклом);
* с дефицитом маркетинговых коммуникаций с интернет-магазином.

### Подготовка к анализу признаков

#### формирование датафрейма X_test_df для изучения признаков

Данный датафрейм собран на основе тестового датафрейма **test_new** (см.5.2.7) и датафрейма **y_test**, содержащего вероятности присвоения целевому признаку 1 (снижение покупательской активности) и столбец 'прогноз_снижения' с информаацией о прогнозируемой активности.

#### Функция построения диаграмм рассеяния c разделением по целевому признаку

In [ ]:
# СОЗДАНИЕ ТАБЛИЦЫ С ПРОГНОЗНЫМИ ЗНАЧЕНИЯМИ
# первая категория относится к 'прогноз_снижения'=1 (снижение активности)
# вторая категория относится к 'прогноз_снижения'=0 (активность сохранится)

X_test_df = pd.DataFrame(data=y_test, columns=['покупательская_активность'])

# добавление признака - вероятности снижения покупательской активности 
# для объектов тестовой выборки на основе лучшей модели (test_new из п.5.2.7)
X_test_df['probability'] = randomized_search.best_estimator_['models'].predict_proba(test_new)[:,1]

# добавление столбца с классификацией целевого признака по пороговому значению 0,5
X_test_df['прогноз_снижения'] = X_test_df['probability'].apply(lambda x: 1 if x>= 0.5 else 0)

# добавление признака прибыльности по покупателям
X_test_df = X_test_df.join(money_test)

# присоединение столбца test_new (взят из 5.2.7)
X_test_df = X_test_df.join(test_new)

# переименования для удобства при визуализации
X_test_df['покупательская_активность'] = X_test_df['покупательская_активность'].apply(lambda x: 'снижение' if x==1 else 'преж_уровень')
X_test_df['прогноз_снижения'] = X_test_df['прогноз_снижения'].apply(lambda x: 'снижение' if x==1 else 'преж_уровень')

X_test_df.head(3) # контроль создания датафрейма

In [ ]:
# функция построения диаграмм рассеяния
def compare_plot(feature_1, feature_2, feature_3):
    plt.figure(figsize=(8, 6))
    plt.title(
        f'Зависимость признака "{feature_2}" от "{feature_1}" с разделением по признаку "{feature_3}"', 
        fontsize=12)
    plt.ylabel(feature_2, fontsize=10)
    plt.xlabel(feature_1, fontsize=10)
    sns.scatterplot(data=segm_df, 
                    x=feature_1, 
                    y=feature_2, 
                    hue=feature_3)
    plt.show()

#### Функция построения диаграмм влияния каждого признака на отдельные объекты (метод SHAP)

In [ ]:
# функция построения диаграмм влияния признаков на целевой признак
def shap_fun(ind):
    for i in ind:
        print('Покупатель под индексом', i)
        shap.plots.waterfall(shap_values[i])

### Анализ сегмента клиентов с максимальной долей покупок по акции и высокой вероятностью снижения покупательской активности

#### Выделение сегмента с высокой вероятностью снижения покупательской активности

In [ ]:
# определение границы сегмента по признаку 'probability'
segm_df = X_test_df

# вызов функции построения диаграммы рассеяния
compare_plot('probability', 'прибыль', 'покупательская_активность')

In [ ]:
# формирование сегмента с высокими значениями 'probability'
segm_df = X_test_df.query('probability > 0.8')

# вызов функции построения диаграммы рассеяния
compare_plot('probability', 'прибыль', 'покупательская_активность')

#### Выделение сегмента клиентов с максимальной долей покупок по акции и высокой вероятностью снижения покупательской активности 

In [ ]:
# зависимость долей покупок по акции от вероятности снижения покупательской активности
# вызов функции построения диаграммы рассеяния
compare_plot('probability', 'акционные_покупки', 'покупательская_активность')

In [ ]:
# формирование сегмента  сегмента клиентов с максимальной долей покупок по акции 
# и высокой вероятностью снижения покупательской активности
segm_df = segm_df.query('акционные_покупки > 0.7')

# вызов функции построения диаграммы рассеяния
compare_plot('probability', 'акционные_покупки', 'покупательская_активность')

In [ ]:
# формирование сегмента  сегмента клиентов с максимальной долей покупок по акции,
# (при высокой вероятности снижения покупательской активности)
# в зависиости от количества неоплаченных товаров
segm_df = segm_df.query('акционные_покупки > 0.7')

# вызов функции построения диаграммы рассеяния
compare_plot('отказов_за_квартал', 'акционные_покупки', 'покупательская_активность')

#### Анализ влияния отдельных признаков на классификацию целевого признака 

In [ ]:
# нахождение индексов по объектам сегмента
segm_ind = segm_df.index

# вывод соответствующих индексов
print('Перечень индексов строк для рассматриваемого сегмента:')
display(segm_ind)

In [ ]:
# влияние признаков на типичные объекты анализа (10 первых индексов для примера)
list_act = [10,  21,  22,  28,  34,  51,  66,  80,  84,  97]

print('ДИАГРАММЫ ВЛИЯНИЯ ОТДЕЛЬНЫХ ПРИЗНАКОВ НА ЦЕЛЕВОЙ ДЛЯ ОБЪЕКТОВ СЕГМЕНТА')
print()

# функция построения диаграмм влияния отдельных признаков
shap_fun(list_act)

*Результат.*

Характеристики покупателей сегмента:
* высокая активность участия в акциях компании;
* значительное количество неоплаченных товаров в корзине за последние 3 месяца;
* низкие среднемесячные значение маркетинговых коммуникаций компании с покупателями за последние 6 месяцев;
* низкие значения среднего количества категорий, просмотренных покупателем за визит в течение последнего месяца;
* низкие значения времени, проведенного на сайте.

Данный сегмент представляет собой покупателей, краткосрочно пользующихся интернет-ресурсом сайта для покупки товаров в период проведения акций. При прекращении проведения акций интерес к интернет-магазину у этой группы покупателей падает, снижается покупательская активность, о чем свидетельствуют: 1) низкие значения среднего количества категорий, просмотренных покупателем за визит в течение последнего месяца; 2) низкие значения времени, проведенного на сайте.

Одним из ключеввых признаков, существенно влияющих на снижение активности покупателей из данного сегмента, являются низкие маркетинговые коммуникации копмпании с ними (число рассылок, звонков, показов рекламы и прочего).

Практически по всем покупателям рассматриваемого сегмента отмечаются случаи неоплаты некоторых товаров. Предположительно данное обстоятельство связано с тем, что пользователи ресурса заказывают товары по акциям с избытком для выбора (ассортимента) и в пункте выдачи отказываются от части товаров. Значительное количество отказов от оплаты влечет за собой для компании дополнительные издержки, связанные с возвратом товаров на склады из пунктов выдачи. 

*Рекомендации заказчику:*

1.Разработка мероприятий по активизации маркетинговых коммуникаций в отношении покупателей из данного сегмента.

2.Расширение ассортимента за счет товаров с более низкими по сранению с имеющимися ценовыми уровнями, расчитанными на рассматриваемых покупателей.

3.С целью сокращения издержек компании по возврату неоплаченных товаров предусмотреть для данного сегмента покупателей ограничение количества заказываемых позиций по акциям.


### Анализ сегмента клиентов, покупающих товары с длинным жизненным циклом

#### Выделение сегмента покупателей товаров с длительным жизненным циклом (техника и посуда)

In [ ]:
# выделение сегмента покупателей, приобретающих только товары длительного жизненного цикла
segm_df = X_test_df.query('((бытовая_техника_и_эл_ка == 1) or (кухонная_посуда == 1) or \
(тех_красоты_и_здоровья == 1)) and ((косметика_и_аксесуары == 0) or (товары_для_детей ==0))')

# вызов функции построения диаграммы рассеяния
compare_plot('probability', 'прибыль', 'покупательская_активность')

In [ ]:
# формирование сегмента с высокими значениями 'probability' (признак снижения пок. активности)
segm_df = segm_df.query('probability > 0.8')

# вызов функции построения диаграммы рассеяния
compare_plot('probability', 'прибыль', 'покупательская_активность')

#### Анализ влияния отдельных признаков на классификацию целевого признака 

In [ ]:
# нахождение индексов по объектам сегмента
segm_ind = segm_df.index

# вывод индексов по выбранным объектам
print('Перечень индексов строк для рассматриваемого сегмента:')
display(segm_ind)

In [ ]:
# влияние признаков на типичные объекты анализа (10 первых индексов для примера)
list_act = [10,  19,  21,  22,  34,  51,  52,  62,  80, 163]

print('ДИАГРАММЫ ВЛИЯНИЯ ОТДЕЛЬНЫХ ПРИЗНАКОВ НА ЦЕЛЕВОЙ ДЛЯ ОБЪЕКТОВ СЕГМЕНТА')
print()

# функция построения диаграмм влияния отдельных признаков
shap_fun(list_act)

*Результат.*

Характеристики покупателей сегмента:
* высокая активность участия в акциях компании;
* значительное количество неоплаченных товаров в корзине за последние 3 месяца;
* низкие средние значения количества страниц, которые просмотрели покупатели за один визит на сайт за последние 3 месяца.

Среди покупателей товаров с длительным жизненным циклом преобладают активные пользователи акционных покупок, целеноправленно выбирающие группы товаров со скидками (о чем свидетельствуеют незначительные количества посещаемых страниц сайта магазина).

*Рекомендации заказчику:*

1.Доработка интерфейса сайта с целью поддержания интереса пользователей сайта к ресурсу, например:
* закрепления на страницах с предлагаемыми по акции товарами ссылок на другие страницы сайта с потенциально интересными для покупателей данного сегмента предложениями;
* публикация на страницах с акционными товарами объявлений о проведении следующих акций и т.д.

2.С целью снижения количества неоплаченных товаров анализ необходимости доработки интерфейса формирования заказа товаров.

## Выводы

В задачу проекта входила разработка решения, позволяющего персонализировать предложения постоянным клиентам интернет-магазина для увеличения их покупательской активности. Для чего построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца. Используя данные модели необходимо выделить сегменты покупателей и разработать для них персонализированные предложения.

Исходные данные для работы находились в нескольких таблицах, содержащих информацию по следующим группам признаков:
* коммуникации с клиентом;
* продуктивное поведение;
* поведение на сайте;
* финансовое поведение.

Для удобства работы над проектом данные были сведены в один датафрейм.

В ходе предобработки данных осуществлялась их проверка на наличие пропусков, полных дубликатов, наличие неявных дубликатов в категориальных признаках. Были обнаружены и устранены неявные дубликаты.

В процессе исследовательского анализа данных были выявлены и устранены аномалии (выбросы) в данных (путем замены на медианные значения). Также сделан вывод о необходимости масштабирования значений исходных данных и стратификации целевого признака.

Для поиска лучшей модели использовался пайплайн. Во время подготовки данных применялся **ColumnTransformer**. Количественные и категориальные признаки обрабатывались в пайплайне раздельно. Для кодирования категориальных признаков были использованы два кодировщика (**OneHotEncoder** и **LabelEncoder**), для масштабирования количественных — два скейлера (**StandardScaler** и **MinMaxScaler**). 

Обучались четыре модели: **DecisionTreeClassifier(), LogisticRegression() и  SVC()**. Для каждой из них подберался как минимум один гиперпараметр. Для задачи была выбрана метрика **roc_auc**, ее выбор аргументирован в выводах к разделу 5 (п.5.3). Данная метрика использовалась при подборе гиперпараметров.

В качестве лучшей была выбрана модель: **LogisticRegression(C=2, penalty='l1', random_state=42, class_weight='balanced', solver='saga').**

После выбора лучшей модели была произведена ее доработка на предмет повышения качества модели:
* добавление в модель настройки **class_weight**;
* подбор оптимальных значений метрик качества **(precision, recall, f1)**.

Все результаты работы детально изложены в ходе проекта, подробные выводы к разделам сведены в **п.1.3, п.2.4, п.3.4, п.4.3, п.5.3, п.6.4, п.7.2.3, п.7.3.2.**

В ходе проекта были установлены следующие важные признаки:

* **'отказов_за_квартал'** - общее число неоплаченных товаров в корзине за последние 3 месяца;
* **'страниц_за_визит'** - среднее количество страниц, которые просмотрел покупатель за один визит на сайт за 3 месяца;
* **'сред_просм_за_визит'** -  сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца;
* **'акционные_покупки'** - среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев;
* **'время_пред_месяц'** - время на сайте в минутах, зафиксированное за предыдущий месяц;
* **'маркет актив 6 мес'** -  среднемес. значение маркетинг. коммуникаций, которое приходилось на покупателя за 6 месяцев;
* **'время_тек_месяц'** - время на сайте в минутах, зафиксированное за предыдущий месяц;
* **'бытовая техника и эл-ка'** - один из классов популярной категории товаров;
* **'выручка_препред_месяц'** - траты покупателей за препредыдущий месяц;
* **'выручка_пред_месяц'** - траты покупателей за предыдущий месяц;
* **'тех красоты и здоровья'** - один из классов популярной категории товаров;
* **'выручка_тек_месяц'** - траты покупателей за предыдущий месяц.

В заключении проекта **(раздел 7)**  производилась сегментация покупателей. Используя результаты моделирования и данные о прибыльности покупателей проведилось графическое и аналитическое исследование групп покупателей.

Анализировалась группа клиентов с максимальной долей покупок по акции и высокой вероятностью снижения покупательской активности. При анализе данного сегмента покупателей, проявляющих высокую активность в акциях компании по льготной продаже товаров, было установлено, что подавляющее большинство представителей этой группы предположительно являются эпизодическими (временными) пользователями услуг интернет-магазина, заинтересованными в бюджетных покупках. По ним зафиксирована слабая маркетинговая коммуникация компании, низкая посещаемость сайта при высоком уровне отказов от покупок. Из этого сделано предположение о наличии узкого ассортимента товаров для данной целевой аудитории. В связи с чем, руководству компании предлагается провести исследование ценовых предложений по всем группам товаров и, при необходимости, расширить их ассортимент. После чего предусмотреть пути активизации маркетинговых мероприятий для более активного предложения товаров данным покупателям.

При анализе сегмента пользователей, приобретающих только товары длительного жтзненного цикла, было установлено, что в подавляющем большинстве данные товары покупаются ими по акциям.

**Рекомендации руководству компании "В один клик"!**

1.Разработка мероприятий по активизации маркетинговых коммуникаций в отношении покупателей рассмотренных сегментов.

2.Расширение ассортимента интернет-магазина за счет товаров с более низкими по сранению с имеющимися ценовыми уровнями.

3.С целью сокращения издержек компании по возврату неоплаченных по акциям товаров предусмотреть для рассмотренных сегментов покупателей ограничение количества заказываемых позиций по акциям.

4.Доработка интерфейса сайта с целью поддержания интереса вышеуказанных пользователей сайта к ресурсу, например:
* закрепления на страницах с предлагаемыми по акции товарами ссылок на другие страницы сайта с потенциально интересными для этих покупателей предложениями;
* публикация на страницах с акционными товарами объявлений о проведении следующих акций и т.д.

5.Расширение состава признаков и проведение дополнительных исследований на предмет установления причин неоплаты заказанных товаров.